# Weather Project by Stefano Chiesa

This project was made by **Chiesa Stefano**, as an assignment for *Python class*.
Resources used: Stackoverflow, ChatGPT, Python Lectures and Geopandas documentation.

**Original tasks**:
The dataset reports the temperature recorded in major cities around the world since 1750. Using this data, the project will need to provide an effective graphical visualization of the change in temperatures over time, highlighting the cities where the largest temperature ranges were recorded during different historical periods. For visualization of the data on a map, see geopandas.

The program will also suggest, depending on the period considered, the best route to follow for a traveler who intends to move from Beijing to Los Angeles by moving step by step to the warmest city among the 3 closest to him.

**Modifications**:
The program provides the best route between two given cities, not just between Beijing (Peking in the dataset) and Los Angeles. I also coded some functions during the testing of the Geopandas package that I kept in the final project.